<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_2A/main_EEGNet_NSL_2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/sagihaider/EEG_Deep.git

fatal: destination path 'EEG_Deep' already exists and is not an empty directory.


In [2]:
import scipy.io as spio
import numpy as np
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

!pip install neural_structured_learning
import neural_structured_learning as nsl
import tensorflow as tf

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [4]:
from numpy import zeros
K.clear_session()

# Initialize parameter
h_cut = [40]
drop_out = [0.5]
k_len = [32, 64, 128]
n_epochs = 300

out_f_name = 'acc_NSL_EEGNet' + '_epochs_' + str(n_epochs) + '_filter_' + str(h_cut) +'_drop_' + str(drop_out) +'_2AData.npy'
print(out_f_name)

nsub = 9
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
loss_score = zeros([nsub, nfilt,ndrop,nkl])
categorical_crossentropy_score = zeros([nsub, nfilt,ndrop,nkl])
categorical_accuracy = zeros([nsub, nfilt,ndrop,nkl])

X_tr = np.empty([288, 22, 1875])
X_ts = np.empty([288, 22, 1875])

for sub_idx, x in enumerate(range(1,10)):
    for h_indx, h in enumerate(h_cut):
      fName = 'EEG_Deep/Data2A/Data_A0' + str(x) + 'T.mat'  # Load Data
      print(fName)
      mat = spio.loadmat(fName)
      r_X_tr = mat['cleanRawEEGData']
      y_tr = mat['cleanClassLabels']
      y_tr = y_tr.flatten() 
  
      print(np.shape(r_X_tr))
      print(np.shape(y_tr))

      for t in range(r_X_tr.shape[0]):
        tril = r_X_tr[t,:,:]
        # tril = tril.transpose()
        tril_filtered = butter_bandpass_filter(tril, lowcut=4, highcut=h, fs=250, order=4)
        # tril_filtered = tril_filtered.transpose()
        X_tr[t,:,:] = tril_filtered 

      # split data of each subject in training and validation
      X_train      = X_tr[0:240,:,500:1250]
      Y_train      = y_tr[0:240]
      X_val       = X_tr[241:,:,500:1250]
      Y_val       = y_tr[241:]

      print(np.shape(X_train))
      print(np.shape(Y_train))
      print(np.shape(X_val))
      print(np.shape(Y_val))

      # convert labels to one-hot encodings.
      Y_train      = np_utils.to_categorical(Y_train-1)
      Y_val       = np_utils.to_categorical(Y_val-1)

      kernels, chans, samples = 1, 22, 750
      # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
      X_val       = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
      print('X_train shape:', X_train.shape)
      print(X_train.shape[0], 'train samples')
      print(X_val.shape[0], 'val samples')

      # Load test data         
      fName = 'EEG_Deep/Data2A/Data_A0' + str(x) + 'E.mat'  # Load Data
      print(fName)
      mat = spio.loadmat(fName)
      r_X_ts = mat['cleanRawEEGData']
      y_ts = mat['cleanClassLabels']

      print(np.shape(r_X_ts))
      print(np.shape(y_ts))

      for t in range(r_X_ts.shape[0]):
        tril = r_X_ts[t,:,:]
        # tril = tril.transpose()
        tril_filtered = butter_bandpass_filter(tril, lowcut=4, 
                                               highcut=40, fs=250, order=4)  
        
      # tril_filtered = tril_filtered.transpose()
      X_ts[t,:,:] = tril_filtered 

      X_test      = X_ts[:,:,500:1250]
      Y_test      = y_ts[:]
      print(np.shape(X_test))
      print(np.shape(Y_test))

      #convert labels to one-hot encodings.
      Y_test      = np_utils.to_categorical(Y_test-1)

      # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

      print('X_train shape:', X_test.shape)
      print(X_test.shape[0], 'train samples')

      for id_d, d in enumerate(drop_out):
        for id_kl, kl in enumerate(k_len):
          print(id_kl, id_d)
          # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
          # model configurations may do better, but this is a good starting point)
          model = EEGNet(nb_classes = 4, Chans = 22, Samples = 750, 
                         dropoutRate = d, kernLength = kl, F1 = 8, 
                         D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')

          adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.5, adv_grad_norm='infinity')
          adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)
  
          # compile the model and set the optimizers
          adv_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
          batch_size = 32

          X_train = tf.cast(X_train, tf.float32)
          X_test = tf.cast(X_test, tf.float32)
          X_val = tf.cast(X_val, tf.float32)


          train_data = tf.data.Dataset.from_tensor_slices({'input': X_train, 'label': Y_train}).batch(batch_size)
          val_data = tf.data.Dataset.from_tensor_slices({'input': X_val, 'label': Y_val}).batch(batch_size)
          test_data = tf.data.Dataset.from_tensor_slices({'input': X_test, 'label': Y_test}).batch(batch_size)

          val_steps = X_val.shape[0] // batch_size

  
          # adv_model.fit(train_data, validation_data=val_data, validation_steps=None, epochs=500, verbose=1)

          history = adv_model.fit(train_data, validation_data=val_data, 
                                  validation_steps=None, epochs=n_epochs, verbose=1)
          
          print('\n# Evaluate on test data')
          results = adv_model.evaluate(test_data)
          print('test loss, test acc:', results)

          loss_score[sub_idx,h_indx,id_d, id_kl] = results[0]
          categorical_crossentropy_score[sub_idx,h_indx,id_d, id_kl] = results[1]
          categorical_accuracy[sub_idx,h_indx,id_d, id_kl] = results[2]

          del model

#           del train_data, val_data, test_data

          # from keras import backend as K 
          # K.clear_session()

np.save(out_f_name, categorical_accuracy)




acc_NSL_EEGNet_epochs_300_filter_[40]_drop_[0.5]_2AData.npy
EEG_Deep/Data2A/Data_A01T.mat
(288, 22, 1875)
(288,)
(240, 22, 750)
(240,)
(47, 22, 750)
(47,)
X_train shape: (240, 1, 22, 750)
240 train samples
47 val samples
EEG_Deep/Data2A/Data_A01E.mat
(288, 22, 1875)
(288, 1)
(288, 22, 750)
(288, 1)
X_train shape: (288, 1, 22, 750)
288 train samples
0 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 8 steps, validate on 2 steps
Epoch 1/300
8/8 [==============================] - 2s 229ms/step - loss: 1.3878 - categorical_crossentropy: 1.3878 - categorical_accuracy: 0.2792 - val_loss: 1.3883 - val_categorical_crossentropy: 1.3883 - val_categorical_accuracy: 0.2128
Epoch 2/300
8/8 [==============================] - 0s 14ms/step - loss: 1.3731 - categorical_crossentropy: 1.3731 - categorical_accuracy: 0.2500 - val_loss: 1.3873 - val_categorical_crossentropy: 1.3873 - val_categorical_accuracy: 0.2766
Epoch 3/300
8/8 [==============================]